In [7]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Input widgets
ticker_input = widgets.Text(
    placeholder='Enter tickers (comma-separated)',
    description='Tickers:',
    layout=widgets.Layout(width='400px')
)

start_date = widgets.DatePicker(
    description='Start Date:'
)

end_date = widgets.DatePicker(
    description='End Date:'
)

# Fetch button
fetch_button = widgets.Button(description='Fetch Data', button_style='success')

# Buttons for insights (initially disabled)
trend_button = widgets.Button(description='Show Trends', button_style='info', disabled=True)
correlation_button = widgets.Button(description='Show Correlations', button_style='info', disabled=True)
risk_return_button = widgets.Button(description='Risk vs Return', button_style='info', disabled=True)

# Output area
output = widgets.Output(layout=widgets.Layout(height='600px'))

# Global variable to store fetched data
global_data = {'data': None}

In [10]:
def plot_trends(data):
    """Plot price trends"""
    fig = go.Figure()
    for col in data.columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[col], mode='lines', name=col))
    fig.update_layout(
        title="Price Trends",
        xaxis_title="Date",
        yaxis_title="Adjusted Close Price ($)",
        hovermode='x unified'
    )
    return fig

def plot_correlation(data):
    """Plot correlation heatmap"""
    returns = data.pct_change().dropna()
    corr_matrix = returns.corr()
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=np.round(corr_matrix.values, 2),
        texttemplate="%{text}",
        textfont={"size": 12}
    ))
    fig.update_layout(title="Correlation Matrix")
    return fig

def plot_risk_return(data):
    """Plot risk vs return scatter"""
    returns = data.pct_change().dropna()
    annual_returns = returns.mean() * 252
    annual_vol = returns.std() * np.sqrt(252)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=annual_vol,
        y=annual_returns,
        mode='markers+text',
        text=annual_returns.index,
        textposition="top center",
        marker=dict(size=15, color='blue', opacity=0.6),
        hovertemplate='<b>%{text}</b><br>Volatility: %{x:.3f}<br>Return: %{y:.3f}<extra></extra>'
    ))
    fig.update_layout(
        title="Risk vs Return (Annualized)",
        xaxis_title="Annualized Volatility",
        yaxis_title="Annualized Return",
        showlegend=False
    )
    return fig

In [11]:
def on_fetch_clicked(b):
    """Fetch data when button is clicked - alternative method"""
    with output:
        clear_output()
        print("Fetching data...")
        try:
            # Validate input
            tickers_str = ticker_input.value.strip()
            if not tickers_str:
                raise Exception("Please enter at least one ticker symbol.")
            
            # Process tickers
            tickers = [t.strip().upper() for t in tickers_str.split(',') if t.strip()]
            if not tickers:
                raise Exception("No valid tickers entered.")
            
            # Try multiple approaches to fetch data
            all_data = []
            successful_tickers = []
            
            for ticker in tickers:
                try:
                    # Method 1: Use yf.Ticker for more control
                    stock = yf.Ticker(ticker)
                    
                    if start_date.value and end_date.value:
                        hist = stock.history(
                            start=start_date.value, 
                            end=end_date.value,
                            auto_adjust=True
                        )
                    else:
                        # Default to 5 years if dates not specified
                        hist = stock.history(period="5y", auto_adjust=True)
                    
                    if not hist.empty:
                        # Use Adjusted Close if available, otherwise Close
                        if 'Close' in hist.columns:
                            close_data = hist['Close'].rename(ticker)
                            all_data.append(close_data)
                            successful_tickers.append(ticker)
                            print(f"✓ Successfully fetched data for {ticker}")
                        else:
                            print(f"✗ No price data found for {ticker}")
                    else:
                        print(f"✗ No historical data found for {ticker}")
                        
                except Exception as e:
                    print(f"✗ Error fetching {ticker}: {str(e)}")
            
            if not all_data:
                raise Exception(f"No data could be fetched for any tickers: {tickers}")
            
            # Combine all data
            combined_data = pd.concat(all_data, axis=1)
            combined_data = combined_data.dropna()
            
            if combined_data.empty:
                raise Exception("No valid data after combining and cleaning")
            
            global_data['data'] = combined_data
            
            # Enable analysis buttons
            trend_button.disabled = False
            correlation_button.disabled = False
            risk_return_button.disabled = False
            
            print(f"Data fetched successfully for {successful_tickers}!")
            print(f"Data range: {combined_data.index[0]} to {combined_data.index[-1]}")
            print(f"Data shape: {combined_data.shape}")
            
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            # Disable buttons if fetch fails
            trend_button.disabled = True
            correlation_button.disabled = True
            risk_return_button.disabled = True

def display_plotly_figure(fig):
    """Properly display Plotly figures in widget output"""
    from IPython.display import display
    
    # Convert to FigureWidget for better Jupyter integration
    fig_widget = go.FigureWidget(fig)
    display(fig_widget)

def on_trend_clicked(b):
    """Show trends using cached data"""
    with output:
        clear_output()
        if global_data['data'] is not None and not global_data['data'].empty:
            try:
                fig = plot_trends(global_data['data'])
                display_plotly_figure(fig)
            except Exception as e:
                print(f"Error plotting trends: {e}")
        else:
            print("Please fetch data first!")

def on_correlation_clicked(b):
    """Show correlations using cached data"""
    with output:
        clear_output()
        if global_data['data'] is not None and not global_data['data'].empty:
            try:
                fig = plot_correlation(global_data['data'])
                display_plotly_figure(fig)
            except Exception as e:
                print(f"Error plotting correlations: {e}")
        else:
            print("Please fetch data first!")

def on_risk_return_clicked(b):
    """Show risk-return using cached data"""
    with output:
        clear_output()
        if global_data['data'] is not None and not global_data['data'].empty:
            try:
                fig = plot_risk_return(global_data['data'])
                display_plotly_figure(fig)
            except Exception as e:
                print(f"Error plotting risk-return: {e}")
        else:
            print("Please fetch data first!")

# Attach callbacks
fetch_button.on_click(on_fetch_clicked)
trend_button.on_click(on_trend_clicked)
correlation_button.on_click(on_correlation_clicked)
risk_return_button.on_click(on_risk_return_clicked)

In [12]:
# Create input layout
input_layout = widgets.VBox([
    ticker_input,
    widgets.HBox([start_date, end_date]),
    fetch_button,
    widgets.HBox([trend_button, correlation_button, risk_return_button])
])

# Display interface
display(input_layout)
display(output)

Output(layout=Layout(height='600px'))